In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ChatVectorDBChain
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredFileLoader
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["QDRANT_API_KEY"] = os.getenv("QDRANT_API_KEY")
os.environ["QDRANT_HOST"] = os.getenv("QDRANT_HOST")

In [8]:
where_the_dir_is = r'D:\Downloaded\BibleBuddy Docs\ASV'
# where_the_dbs_will_be_saved = r'D:\Downloaded\BibleBuddy\db\Bibles\Collections\KJV'

In [9]:
# loader_cls = lambda path: UnstructuredFileLoader(path)
# This will allow whole directorys to be embedded. 
loader = DirectoryLoader(where_the_dir_is)
# loader_cls = lambda path: UnstructuredFileLoader(path, mode="elements", max_tokens=2000)

# loader.loader_cls = lambda path: UnstructuredFileLoader(path, mode="elements")
docs = loader.load()
len(docs)

1

In [10]:
documents = loader.load()

In [ ]:
# from langchain.document_loaders import TextLoader
# loader = TextLoader('db/paul_graham_essay.txt')
# documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

In [ ]:
# host = "biblebuddycluster"
api_key = os.environ["QDRANT_API_KEY"]
hosturl = os.environ["QDRANT_HOST"]
collectionname = 'ASV'
qdrant = Qdrant.from_documents(docs, embeddings, url=hosturl, prefer_grpc=True, api_key=api_key, collection_name=collectionname)


In [7]:
query = "What did Jesus say about Sabbath?"
docs = qdrant.similarity_search(query)
docs[0]

Document(page_content='24 And the Pharisees said to Him, "Look, why do they do what is not lawful on the Sabbath?"\n\n25 But He said to them, "Have you never read what David did when he was in need and hungry, he and those with him:\n\n26 how he went into the house of God in the days of Abiathar the high priest, and ate the showbread, which is not lawful to eat except for the priests, and also gave some to those who were with him?"\n\n27 And He said to them, "The Sabbath was made for man, and not man for the Sabbath.\n\n28 Therefore the Son of Man is also Lord of the Sabbath."\n\n3 And He entered the synagogue again, and a man was there who had a withered hand.\n\n2 So they watched Him closely, whether He would heal him on the Sabbath, so that they might accuse Him.\n\n3 And He said to the man who had the withered hand, "Step forward."\n\n4 Then He said to them, "Is it lawful on the Sabbath to do good or to do evil, to save life or to kill?" But they kept silent.', lookup_str='', metad

In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [22]:
system_template="""Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. Provide a scripture reference.
ALWAYS return a "sources" part in your answer.
The "sources" part should be a reference to the source of the document from which you got your answer.

The output should be a markdown code snippet formatted in the following schema:
```json
{{
answer: is foo
sources: xyz
}}
```
Begin!
----------------
{context}"""
messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]
prompt = ChatPromptTemplate.from_messages(messages)

In [23]:
qa = ChatVectorDBChain.from_llm(ChatOpenAI(temperature=0), qdrant,qa_prompt=prompt, return_source_documents=True)

In [24]:
chat_history = []
query = "What is Exodus about?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

'Exodus is the second book of the Bible and it tells the story of how God delivered the Israelites from slavery in Egypt and made a covenant with them at Mount Sinai. It also includes the giving of the Ten Commandments and the construction of the Tabernacle. \n\nSources: Exodus 1-40'

In [25]:
result['source_documents']

[Document(page_content="Exodus 19:1\tIn the third month, when the children of Israel were gone forth out of the land of Egypt, the same day came they into the wilderness of Sinai.\n\nExodus 19:2\tFor they were departed from Rephidim, and were come to the desert of Sinai, and had pitched in the wilderness; and there Israel camped before the mount.\n\nExodus 19:3\tAnd Moses went up unto God, and the LORD called unto him out of the mountain, saying, Thus shalt thou say to the house of Jacob, and tell the children of Israel;\n\nExodus 19:4\tYe have seen what I did unto the Egyptians, and how I bare you on eagles' wings, and brought you unto myself.\n\nExodus 19:5\tNow therefore, if ye will obey my voice indeed, and keep my covenant, then ye shall be a peculiar treasure unto me above all people: for all the earth is mine:\n\nExodus 19:6\tAnd ye shall be unto me a kingdom of priests, and an holy nation. These are the words which thou shalt speak unto the children of Israel.", lookup_str='', 

Testing persistance

In [ ]:
!pip install qdrant_client

In [45]:
# from langchain import Qdrant
from qdrant_client import QdrantClient
from langchain.vectorstores import Qdrant

host = "biblebuddycluster"
api_key = os.environ["QDRANT_API_KEY"]
hosturl = os.environ["QDRANT_HOST"]
client = QdrantClient(url=hosturl, prefer_grpc=True, api_key=api_key )
collection_name = "KJV"
embeddings = OpenAIEmbeddings().
qdrant2 = Qdrant(client, collection_name, embedding_function=embeddings.embed_query)

In [46]:
qdrant2

In [47]:
query = "What did Jesus say about Sabbath?"
docs = qdrant2.similarity_search(query)
docs[0]

Document(page_content='Luke 13:14\tAnd the ruler of the synagogue answered with indignation, because that Jesus had healed on the sabbath day, and said unto the people, There are six days in which men ought to work: in them therefore come and be healed, and not on the sabbath day.\n\nLuke 13:15\tThe Lord then answered him, and said, Thou hypocrite, doth not each one of you on the sabbath loose his ox or his ass from the stall, and lead him away to watering?\n\nLuke 13:16\tAnd ought not this woman, being a daughter of Abraham, whom Satan hath bound, lo, these eighteen years, be loosed from this bond on the sabbath day?\n\nLuke 13:17\tAnd when he had said these things, all his adversaries were ashamed: and all the people rejoiced for all the glorious things that were done by him.\n\nLuke 13:18\tThen said he, Unto what is the kingdom of God like? and whereunto shall I resemble it?', lookup_str='', metadata={'source': 'D:\\Downloaded\\BibleBuddy Docs\\KJV\\kjv.txt'}, lookup_index=0)